# Using Microsoft SEAL on Android

With Xamarin, and the Microsoft SEAL NuGet package, we are able to to develop Android applications incorporating homomorphic encryption using .NET. But before we jump into the Android app development, we will begin with writing a simple C# program that will be transferred to the Android app. This program will consist of repeatedly encrypting, then decrypting a number, and measuring the time taken on average to encrypt and decrypt the number once. This is a very important time to know, as it can be the bottleneck in the frequency a certain operation can run with on a phone.

To begin with, we need to import some libraries that we will be using. We need to get the NuGet packaged for Microsoft SEAL, as well as the System.Diagnostics class which will help us in timing. 

**Note**: Do not include `#r "nuget:Microsoft.Research.SEALNet, 3.5.1"` in actual C# code. That is jupyter specific code for importing the NuGet package. Follow steps on the Microsoft SEAL GitHub to set up an environment to use SEAL with C#/.NET.

In [11]:
#r "nuget:Microsoft.Research.SEALNet, 3.5.1"
using Microsoft.Research.SEAL;

using System;
using System.Diagnostics;

Our simple program will consist of setting up the encryption parameters, then encrypting and decrypting a number inside a for loop. To do this, we simply set up the encryption parameters as is done in the example for the CKKS scheme in the [Microsoft SEAL examples](https://github.com/microsoft/SEAL/blob/master/dotnet/examples/4_CKKS_Basics.cs). We then encode a number, the mathematical constant *e* to the 12th decimal place, in this case, then encrypt it. We immediately decrypt it then decode it back to a floating point number.

In [16]:
test.Main();

public class test{
    public static void Main(){
        using EncryptionParameters parms = new EncryptionParameters(SchemeType.CKKS);
        ulong polyModulusDegree = 8192;
        parms.PolyModulusDegree = polyModulusDegree;
        parms.CoeffModulus = CoeffModulus.Create(
                        polyModulusDegree, new int[]{ 60, 40, 40, 60 });
        double scale = Math.Pow(2.0, 40);

        using SEALContext context = new SEALContext(parms);
        Console.WriteLine();

        using KeyGenerator keygen = new KeyGenerator(context);
        using PublicKey publicKey = keygen.PublicKey;
        using SecretKey secretKey = keygen.SecretKey;
        using Encryptor encryptor = new Encryptor(context, publicKey);
        using Decryptor decryptor = new Decryptor(context, secretKey);

        using CKKSEncoder encoder = new CKKSEncoder(context);

        using Plaintext xPlain = new Plaintext();
        using Plaintext plainResult = new Plaintext();
        using Ciphertext x1Encrypted = new Ciphertext();

        List<double> result = new List<double>();

        encoder.Encode(2.718281828459, scale, xPlain);
        encryptor.Encrypt(xPlain, x1Encrypted);
        decryptor.Decrypt(x1Encrypted, plainResult);
        encoder.Decode(plainResult, result);

        Console.WriteLine("Original: " + (2.718281828459).ToString());
        Console.WriteLine("Enc/Dec:  " + (result[0]).ToString("#.############"));
    }
}


Original: 2.718281828459
Enc/Dec:  2.718281829085


After running this, we can see that the encrypted then decrypted value is accurate to the 8th decimal place, or about 10^(-8). The parameters and sclae we chose gave us about 20 bits of precision after the decimal point. Since 10^(-8) is smaller than 2^(-20) ~ 10^(-7), we were at least as accurate as what our parameters guaranteed us.

After running this to verify that the result is the same (or at least similar) to our original value, we can add code to display the time it took to encrypt and decrypt the number, using the [Stopwatch class](https://docs.microsoft.com/en-us/dotnet/api/system.diagnostics.stopwatch?view=netcore-3.1). We can directly print out the number of milliseconds using the `ElapsedMilliseconds` property, but we will instead use the `ElapsedTicks`, which gives a higher time resolution. We also use the `Frequency` field to convert from ticks to a measure of time.

In our timing of encryption and decryption, we also include the time taken for encoding and decoding, as for any practical purpose, our data is not going to be provided as an encoded polynomial, and we would have to take time to do so.

In [23]:
test.Main();

public class test{
    public static void Main(){
        using EncryptionParameters parms = new EncryptionParameters(SchemeType.CKKS);
        ulong polyModulusDegree = 8192;
        parms.PolyModulusDegree = polyModulusDegree;
        parms.CoeffModulus = CoeffModulus.Create(
                        polyModulusDegree, new int[]{ 60, 40, 40, 60 });
        double scale = Math.Pow(2.0, 40);

        using SEALContext context = new SEALContext(parms);
        Console.WriteLine();

        using KeyGenerator keygen = new KeyGenerator(context);
        using PublicKey publicKey = keygen.PublicKey;
        using SecretKey secretKey = keygen.SecretKey;
        using Encryptor encryptor = new Encryptor(context, publicKey);
        using Decryptor decryptor = new Decryptor(context, secretKey);

        using CKKSEncoder encoder = new CKKSEncoder(context);

        using Plaintext xPlain = new Plaintext();
        using Plaintext plainResult = new Plaintext();
        using Ciphertext x1Encrypted = new Ciphertext();

        List<double> result = new List<double>();
        
        Stopwatch stopWatch = new Stopwatch();
        stopWatch.Start();
        
        encoder.Encode(2.718281828459, scale, xPlain);
        encryptor.Encrypt(xPlain, x1Encrypted);
        decryptor.Decrypt(x1Encrypted, plainResult);
        encoder.Decode(plainResult, result);

        stopWatch.Stop();
        
        Console.WriteLine("Original:   " + (2.718281828459).ToString());
        Console.WriteLine("Enc/Dec:    " + (result[0]).ToString("#.############"));
        Console.WriteLine("Time taken: " + (1000.0*stopWatch.ElapsedTicks/(Stopwatch.Frequency)).ToString() + "ms");
    }
}


Original:   2.718281828459
Enc/Dec:    2.718281828575
Time taken: 14.179ms


An issue that could arise by timing this way could be noisy data, where the time taken is off by a few milliseconds each run due to external factors. Doing the encryption and decryption many times, and finding the average allows us to get a more accurate measurement of the time taken. To accomplish this, we wrap our encryption/decryption in a for loop that runs 1000 times, and divide the time taken by 1000, to get the average time taken.

In [25]:
test.Main();

public class test{
    public static void Main(){
        using EncryptionParameters parms = new EncryptionParameters(SchemeType.CKKS);
        ulong polyModulusDegree = 8192;
        parms.PolyModulusDegree = polyModulusDegree;
        parms.CoeffModulus = CoeffModulus.Create(
                        polyModulusDegree, new int[]{ 60, 40, 40, 60 });
        double scale = Math.Pow(2.0, 40);

        using SEALContext context = new SEALContext(parms);
        Console.WriteLine();

        using KeyGenerator keygen = new KeyGenerator(context);
        using PublicKey publicKey = keygen.PublicKey;
        using SecretKey secretKey = keygen.SecretKey;
        using Encryptor encryptor = new Encryptor(context, publicKey);
        using Decryptor decryptor = new Decryptor(context, secretKey);

        using CKKSEncoder encoder = new CKKSEncoder(context);

        using Plaintext xPlain = new Plaintext();
        using Plaintext plainResult = new Plaintext();
        using Ciphertext x1Encrypted = new Ciphertext();

        List<double> result = new List<double>();
        
        Stopwatch stopWatch = new Stopwatch();
        stopWatch.Start();
        
        for (int i = 0; i < 1000; i++){        
            encoder.Encode(2.718281828459, scale, xPlain);
            encryptor.Encrypt(xPlain, x1Encrypted);
            decryptor.Decrypt(x1Encrypted, plainResult);
            encoder.Decode(plainResult, result);
        }
        stopWatch.Stop();
        
        Console.WriteLine("Original:   " + (2.718281828459).ToString());
        Console.WriteLine("Enc/Dec:    " + (result[0]).ToString("#.############"));
        Console.WriteLine("Time taken: " + (1.0*stopWatch.ElapsedTicks/(Stopwatch.Frequency)).ToString("##.###") + "ms on average per enc/dec");
    }
}


Original:   2.718281828459
Enc/Dec:    2.718281828073
Time taken: 11.631ms on average per enc/dec


We finally have a simple program that gives a fairly accurate time of how long it takes to encrypt and decrypt a single value with some given encryption parameters. We can also see that our average deviates from the single value we calculated before quite a bit, showing the inaccuracy of taking a single measurement.

Since the timing is heavily affected by system specifications, my specs are:

CPU: Intel Core i7-6700HQ @ 2.60 GHz. 1 physical processor; 4 cores; 8 threads

RAM: 16 GB

Motherboard: Lenovo Ideapad Y700 Touch-15ISK/Allsparks 5A2 (LENOVO)

Storage: ATA ST1000LM035-1RK1 (1 TB)

OS: Windows 10 Home (Version 10.0.18363 Build 18363)

## Working with Xamarin 

Now that we have made a working program in base C#, we can move on to transfer it to an Android app and test the time it takes on that. To develop on Android, we will be using [Xamarin](https://dotnet.microsoft.com/apps/xamarin) in Visual Studio 2019, which allows us to easily move from our simple C# above to an Android app that accomplishes the same task. To install Xamarin, follow the steps outlined [here](https://docs.microsoft.com/en-us/xamarin/get-started/installation/windows). 

**Note**: If you do not have Visual Studio 2019, you will be instructed to download it.
**Note**: This will *not* work with Linux, so you will have to use either Mac or Windows - I use Windows 10.

Start by opening Visual Studio 2019, and select "Create a new project" under "Get Started". 

![vs1](img/vs1.png)

Now, in the search bar, search for "Xamarin.forms", and click the option that says "Mobile App (Xamarin.Forms)". We will be using Xamarin forms specifically to create this basic Android app, due to its simplicity.

![vs2](img/vs2.png)

In the next window, you will get an option to name your project and set the location. I use the default location, and name the project "EncDecTiming". We then create the a blank project.

![vs3](img/vs3.png)
![vs4](img/vs4.png)

After some time to set up the project, you will arrive at the regular Visual Studio window. On the right side of the screen, you should see the Solution Explorer. Immediately under the Solution Explorer, right click on "EncDecTiming". and select "Manage NuGet Packages"

![vs5](img/vs5.png)



After, click "Browse", and search for "Microsoft.Research.SEALNet", and install version 3.5.1. You may get a popup asking you to click "OK" to proceed with the installation.

![vs6](img/vs6.png)

Once it finishes installing, you can close that tab. Under EncDecTiming.Android, right click "References", and select "Manage NuGet Packages..." again. Follow the same steps as above to install SEAL for Android as well.

![vs6](img/vs61.png)

For the next steps, you will need an Android phone. It is possible to deploy the app to a virtual Android machine, but on Windows you would need to enable HyperV to do so. HyperV is unavailable on Windows 10 Home, so I could not test it, so the remaining steps are for deploying to a physical Android device.

To configure your Android device, do the following:
1. Open settings
2. Search for "Build Number", or go to "About Phone" (usually at the bottom) -> "Software information" -> "Build Number"
3. Click "Build Number" repeatedly until "Developer Options" are enabled
4. Search for "USB Debugging", or go to "Developer Options" (now at the bottom of Settings) -> "USB Debugging" (under "Debugging")
5. Enable "USB Debugging"

Then, after your device is configured, we can allow our computer to deploy to the device.
1. Plug in device to computer using USB cable
2. Allow USB Debugging in the pop up that shows up on your phone
3. Check "Always allow from this computer"
4. Click "Allow"

After doing these steps, you should see your device in the top task bar next to a green run button. 

![vs7](img/vs7.png)

The device I am using is a Samsung A5 runnning Android 8, which is exactly what shows up. Click the run button to begin debugging and run the default empty Xamarin form program, to test if the compiling and deploying works. The first time you do this may take longer than usual. After it has finished deploying, you should see something like the following on your phone:

<img src="img/vs8.png" width="400">

The default program is an empty form with a single label in the center. After verifying that the app deployed correctly, select "Stop Debugging" from "Debug".

In order to make our app, we will have to modify two files, `MainPage.xaml` and `MainPage.xaml.cs`, which can be opened from Solution Explorer.

<img src="img/vs9.png" width="400">


First, we start by modifying `MainPage.xaml` to include a button and a modifiable label. We will run our encryption/decryption program every time the button is pressed, and display the output in the label. The default `MainPage.xaml` looks like the following:

```
<?xml version="1.0" encoding="utf-8" ?>
<ContentPage xmlns="http://xamarin.com/schemas/2014/forms"
             xmlns:x="http://schemas.microsoft.com/winfx/2009/xaml"
             xmlns:d="http://xamarin.com/schemas/2014/forms/design"
             xmlns:mc="http://schemas.openxmlformats.org/markup-compatibility/2006"
             mc:Ignorable="d"
             x:Class="EncDecTiming.MainPage">

    <StackLayout>
        <!-- Place new controls here -->
        <Label Text="Welcome to Xamarin.Forms!" 
           HorizontalOptions="Center"
           VerticalOptions="CenterAndExpand" />
    </StackLayout>

</ContentPage>
```

We will modify it to

```
<?xml version="1.0" encoding="utf-8" ?>
<ContentPage xmlns="http://xamarin.com/schemas/2014/forms"
             xmlns:x="http://schemas.microsoft.com/winfx/2009/xaml"
             xmlns:d="http://xamarin.com/schemas/2014/forms/design"
             xmlns:mc="http://schemas.openxmlformats.org/markup-compatibility/2006"
             mc:Ignorable="d"
             x:Class="EncDecTiming.MainPage">

    <StackLayout>
        <!-- Place new controls here -->
        <Label Text="{Binding LabelText}" 
           HorizontalOptions="Center"
           VerticalOptions="CenterAndExpand"/>
        <Button x:Name="btn" Text="Click To Run" Clicked="Handle_Clicked" />
    </StackLayout>

</ContentPage>

```

For the label, we bind the value of the label to a a string, `LabelText`, which we will get to later. We also make a button with default text, and a function that will handle what happens when the butotn is clicked (`Handle_Clicked`).

Now moving to `MainPage.xaml.cs`, we see the following libraries included by default

In [ ]:
using System;
using System.Collections.Generic;
using System.ComponentModel;
using System.Linq;
using System.Text;
using System.Threading.Tasks;
using Xamarin.Forms;

We will add the libraries for SEAL and Stopwatch

In [28]:
using System.Diagnostics;
using Microsoft.Research.SEAL;

The rest of the file is

In [ ]:
namespace EncDecTiming
{
    // Learn more about making custom code visible in the Xamarin.Forms previewer
    // by visiting https://aka.ms/xamarinforms-previewer
    [DesignTimeVisible(false)]
    public partial class MainPage : ContentPage
    {
        public MainPage()
        {
            InitializeComponent();
        }
    }
}

We start by adding our string for the label in the class, with getters and setters. We use getters and setters in order to be able to notify that there was a change in a property, so the label can update when the string is set.

In [ ]:
private string labelText;
public string LabelText
{
    get { return labelText; }
    set
    {
        labelText = value;
        OnPropertyChanged(nameof(LabelText)); // Notify that there was a change on this property
    }
}

We also modify the constructor to indicate where the binded value is, and to initialize the label.

In [ ]:
public MainPage()
{
    InitializeComponent();
    BindingContext = this;

    LabelText = "Hello, press button to start";
}

Lastly, we make our button handler method, `void Handle_Clicked`. This method will be the exact same as `test.Main()` from our original C# code to test the time it took to encrypt/decrypt, but without the lines to write to the console. Instead, we will change the label text to show our output.

In [ ]:
void Handle_Clicked(object sender, System.EventArgs e)
{
    using EncryptionParameters parms = new EncryptionParameters(SchemeType.CKKS);
    ulong polyModulusDegree = 8192;
    parms.PolyModulusDegree = polyModulusDegree;
    parms.CoeffModulus = CoeffModulus.Create(
                    polyModulusDegree, new int[] { 60, 40, 40, 60 });
    double scale = Math.Pow(2.0, 40);

    using SEALContext context = new SEALContext(parms);
    Console.WriteLine();

    using KeyGenerator keygen = new KeyGenerator(context);
    using PublicKey publicKey = keygen.PublicKey;
    using SecretKey secretKey = keygen.SecretKey;
    using Encryptor encryptor = new Encryptor(context, publicKey);
    using Decryptor decryptor = new Decryptor(context, secretKey);

    using CKKSEncoder encoder = new CKKSEncoder(context);

    using Plaintext xPlain = new Plaintext();
    using Plaintext plainResult = new Plaintext();
    using Ciphertext x1Encrypted = new Ciphertext();

    List<double> result = new List<double>();

    Stopwatch stopWatch = new Stopwatch();
    stopWatch.Start();

    for (int i = 0; i < 1000; i++)
    {
        encoder.Encode(2.718281828459, scale, xPlain);
        encryptor.Encrypt(xPlain, x1Encrypted);
        decryptor.Decrypt(x1Encrypted, plainResult);
        encoder.Decode(plainResult, result);
    }
    stopWatch.Stop();

    LabelText = "Original: " + (2.718281828459).ToString() + "\nEnc/Dec: " + (result[0]).ToString("#.############") + "\nTime taken: " + (1.0 * stopWatch.ElapsedTicks / (Stopwatch.Frequency)).ToString("##.###") + "ms on average per enc/dec";
}

We include the `\n` to add new lines to our label, a simple workaround to instead having 3 labels for the 3 outputs. You may run into an issue where every line with a `using` statement shows an error, telling you to use C# 8.0 instead of C# 7.3. To fix this, open the directory where your solution is located, then open the directory that has the solution name (EncDecTiming in this case). Do *not* open EncDecTiming.Android or EncDecTiming.iOS instead. Inside this directory, open the file with the .csproj extension using a text editor. The file should look like

```
<Project Sdk="Microsoft.NET.Sdk">

  <PropertyGroup>
    <TargetFramework>netstandard2.0</TargetFramework>
    <ProduceReferenceAssembly>true</ProduceReferenceAssembly>
  </PropertyGroup>

  <PropertyGroup Condition="'$(Configuration)|$(Platform)'=='Debug|AnyCPU'">
    <DebugType>portable</DebugType>
    <DebugSymbols>true</DebugSymbols>
  </PropertyGroup>

  <ItemGroup>
    <PackageReference Include="Microsoft.Research.SEALNet" Version="3.5.1" />
    <PackageReference Include="Xamarin.Forms" Version="4.5.0.495" />
    <PackageReference Include="Xamarin.Essentials" Version="1.3.1" />
  </ItemGroup>
</Project>
```

Modify this file to

```
<Project Sdk="Microsoft.NET.Sdk">

  <PropertyGroup>
    <TargetFramework>netstandard2.0</TargetFramework>
    <LangVersion>8.0</LangVersion>
    <Nullable>enable</Nullable>
    <ProduceReferenceAssembly>true</ProduceReferenceAssembly>
  </PropertyGroup>

  <PropertyGroup Condition="'$(Configuration)|$(Platform)'=='Debug|AnyCPU'">
    <DebugType>portable</DebugType>
    <DebugSymbols>true</DebugSymbols>
  </PropertyGroup>

  <ItemGroup>
    <PackageReference Include="Microsoft.Research.SEALNet" Version="3.5.1" />
    <PackageReference Include="Xamarin.Forms" Version="4.5.0.495" />
    <PackageReference Include="Xamarin.Essentials" Version="1.3.1" />
  </ItemGroup>
</Project>
```

which will set the C# version to 8.0

Your final `MainPage.xaml.cs` file should look like

In [ ]:
using System;
using System.Collections.Generic;
using System.ComponentModel;
using System.Linq;
using System.Text;
using System.Threading.Tasks;
using Xamarin.Forms;

using System.Diagnostics;
using Microsoft.Research.SEAL;

namespace EncDecTiming
{
    // Learn more about making custom code visible in the Xamarin.Forms previewer
    // by visiting https://aka.ms/xamarinforms-previewer
    [DesignTimeVisible(false)]
    public partial class MainPage : ContentPage
    {
        public MainPage()
        {
            InitializeComponent();
            BindingContext = this;

            LabelText = "Hello, press button to start";
        }

        private string labelText;
        public string LabelText
        {
            get { return LabelText; }
            set
            {
                LabelText = value;
                OnPropertyChanged(nameof(LabelText)); // Notify that there was a change on this property
            }
        }

        void Handle_Clicked(object sender, System.EventArgs e)
        {
            using EncryptionParameters parms = new EncryptionParameters(SchemeType.CKKS);
            ulong polyModulusDegree = 8192;
            parms.PolyModulusDegree = polyModulusDegree;
            parms.CoeffModulus = CoeffModulus.Create(
                            polyModulusDegree, new int[] { 60, 40, 40, 60 });
            double scale = Math.Pow(2.0, 40);

            using SEALContext context = new SEALContext(parms);
            Console.WriteLine();

            using KeyGenerator keygen = new KeyGenerator(context);
            using PublicKey publicKey = keygen.PublicKey;
            using SecretKey secretKey = keygen.SecretKey;
            using Encryptor encryptor = new Encryptor(context, publicKey);
            using Decryptor decryptor = new Decryptor(context, secretKey);

            using CKKSEncoder encoder = new CKKSEncoder(context);

            using Plaintext xPlain = new Plaintext();
            using Plaintext plainResult = new Plaintext();
            using Ciphertext x1Encrypted = new Ciphertext();

            List<double> result = new List<double>();

            Stopwatch stopWatch = new Stopwatch();
            stopWatch.Start();

            for (int i = 0; i < 1000; i++)
            {
                encoder.Encode(2.718281828459, scale, xPlain);
                encryptor.Encrypt(xPlain, x1Encrypted);
                decryptor.Decrypt(x1Encrypted, plainResult);
                encoder.Decode(plainResult, result);
            }
            stopWatch.Stop();

            LabelText = "Original: " + (2.718281828459).ToString() + "\nEnc/Dec: " + (result[0]).ToString("#.############") + "\nTime taken: " + (1.0 * stopWatch.ElapsedTicks / (Stopwatch.Frequency)).ToString("##.###") + "ms on average per enc/dec";
        }
    }
}

Once again, click the green run button to compile and deploy the app to your phone. Your phone should show the following screen

<img src="img/vs10.png" width="400">

Press "CLICK TO RUN", and wait for the program to finish executing (could take upwards of a minute due to encrypting/decryption 1000 times). After it is done, you should get a screen that looks like

<img src="img/vs11.jpg" width="400">

Unsurprisingly, the program runs slower on a phone than on a computer, but the slow down is almost 7x! This could be a combination of a phone's processor being much weaker and SEAL not being as parallelized. With this output, we have completed making and running a basic Android app using SEAL to time encryption/decryption.